In [1]:
from google.colab import userdata
from huggingface_hub import login
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import json
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm

# --- Paths and Setup ---
# Ensure these paths are correct for your Google Colab environment
IMAGE_FOLDER = "/content/drive/MyDrive/RL_final_project/image/img_all"
EXPLANATION_JSON = "/content/drive/MyDrive/RL_final_project/bdd_for_clip/bdd_all_1a1r.json"
ARCHIVE_PATH = "/content/drive/MyDrive/RL_final_project/bdd_for_clip/img_all.7z"

# --- Unzip Image Archive ---
# This block installs the necessary tool and unzips the images.
print("Preparing to unzip images...")
os.makedirs(IMAGE_FOLDER, exist_ok=True)
print("Installing p7zip utility...")
os.system('apt-get -qq install -y p7zip-full') # Using -qq for quieter installation

if os.path.exists(ARCHIVE_PATH):
    print(f"Found archive. Unzipping {ARCHIVE_PATH} to {IMAGE_FOLDER}...")
    # Using '7z x' which is better at handling paths and extracting to a specific folder.
    # The '-y' flag answers 'yes' to all prompts (e.g., overwrite).
    unzip_command = f'7z x "{ARCHIVE_PATH}" -o"{IMAGE_FOLDER}" -y'
    os.system(unzip_command)
    print("Unzipping complete.")

    # --- DEBUGGING STEP ---
    # List the contents of the image folder to see the actual structure.
    print("\n--- Listing contents of the destination folder for debugging ---")
    os.system(f'ls -R "{IMAGE_FOLDER}" | head -n 10') # Show the first 10 lines of the directory structure
    print("-----------------------------------------------------------------\n")

else:
    print(f"Warning: Archive file not found at {ARCHIVE_PATH}. Assuming images are already in {IMAGE_FOLDER}.")

# Check if the main image directory exists before starting
if not os.path.isdir(IMAGE_FOLDER):
    print(f"Error: The image folder was not found at '{IMAGE_FOLDER}'")
    print("Please ensure the path is correct and your Google Drive is mounted.")
    exit()
if not os.path.exists(EXPLANATION_JSON):
    print(f"Error: JSON file not found at '{EXPLANATION_JSON}'")
    exit()

In [ ]:

# --- Device Setup ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- Load Model and Processor ---
print("Loading CLIP model...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
print("Model loaded.")

# Load the entire dataset from JSON
with open(EXPLANATION_JSON, "r") as f:
    data_to_process = json.load(f)

# --- Create Candidate Labels for Zero-Shot Classification ---
all_actions = sorted(list(set(action for entry in data_to_process for action in entry.get("action", []))))
all_reasons = sorted(list(set(reason for entry in data_to_process for reason in entry.get("reason", []))))
# Create combined labels for action-reason pairs
all_action_reasons = sorted(list(set(f"{entry.get('action', [''])[0]}, because {entry.get('reason', [''])[0]}" for entry in data_to_process if entry.get('action') and entry.get('reason'))))

print(f"Found {len(all_actions)} unique actions.")
print(f"Found {len(all_reasons)} unique reasons.")
print(f"Found {len(all_action_reasons)} unique action-reason pairs.")


def get_prediction_and_probability(image_path, text_candidates, ground_truth_text):
    """
    Performs two tasks:
    1. Predicts the best text candidate from a list.
    2. Calculates the similarity probability for the specific ground_truth_text.
    """
    try:
        image = Image.open(image_path).convert("RGB")
        # To improve efficiency, we process all candidates plus the specific ground truth text at once.
        # This avoids running the model multiple times on the same image.
        all_texts_for_processing = text_candidates[:] # Make a copy
        if ground_truth_text not in all_texts_for_processing:
            all_texts_for_processing.append(ground_truth_text)

        inputs = clip_processor(text=all_texts_for_processing, images=image, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            outputs = clip_model(**inputs)
            logits_per_image = outputs.logits_per_image[0]

            # Task 1: Find the best prediction among the original candidates
            candidate_logits = logits_per_image[:len(text_candidates)]
            best_candidate_index = candidate_logits.argmax().item()
            prediction = text_candidates[best_candidate_index]

            # Task 2: Find the probability for the specific ground truth text
            gt_index = all_texts_for_processing.index(ground_truth_text)
            gt_logit = logits_per_image[gt_index]
            probability = torch.sigmoid(gt_logit).item()

            return prediction, probability
    except Exception as e:
        return None, None

# --- Initialize result and tracking variables ---
results_action_prob = []
results_reason_prob = []
results_action_reason_prob = []

correct_action_predictions = 0
correct_reason_predictions = 0
correct_action_reason_predictions = 0
processed_count = 0

# --- Main Processing Loop ---
for entry in tqdm(data_to_process, desc="Predicting and calculating probabilities"):
    file_name = entry.get("file_name")
    ground_truth_action = entry.get("action", [""])[0]
    ground_truth_reason = entry.get("reason", [""])[0]

    if not file_name or not ground_truth_action or not ground_truth_reason:
        continue

    image_path = os.path.join(IMAGE_FOLDER, file_name)
    if not os.path.exists(image_path):
        continue

    processed_count += 1

    # --- Action Prediction and Probability ---
    predicted_action, action_prob = get_prediction_and_probability(image_path, all_actions, ground_truth_action)
    if predicted_action and predicted_action == ground_truth_action:
        correct_action_predictions += 1
    if action_prob is not None:
        results_action_prob.append({"probability": action_prob})

    # --- Reason Prediction and Probability ---
    predicted_reason, reason_prob = get_prediction_and_probability(image_path, all_reasons, ground_truth_reason)
    if predicted_reason and predicted_reason == ground_truth_reason:
        correct_reason_predictions += 1
    if reason_prob is not None:
        results_reason_prob.append({"probability": reason_prob})

    # --- Combined Action-Reason Prediction and Probability ---
    ground_truth_action_reason = f"{ground_truth_action}, because {ground_truth_reason}"
    predicted_action_reason, action_reason_prob = get_prediction_and_probability(image_path, all_action_reasons, ground_truth_action_reason)
    if predicted_action_reason and predicted_action_reason == ground_truth_action_reason:
        correct_action_reason_predictions += 1
    if action_reason_prob is not None:
        results_action_reason_prob.append({"probability": action_reason_prob})


# --- Final Output ---
print("\n--- [PART 1] Zero-Shot Prediction Accuracy ---")
if processed_count > 0:
    action_accuracy = (correct_action_predictions / processed_count) * 100
    reason_accuracy = (correct_reason_predictions / processed_count) * 100
    action_reason_accuracy = (correct_action_reason_predictions / processed_count) * 100

    print(f"Processed {processed_count} valid image-text pairs.")
    print(f"Action Prediction Accuracy: {action_accuracy:.2f}% ({correct_action_predictions}/{processed_count})")
    print(f"Reason Prediction Accuracy: {reason_accuracy:.2f}% ({correct_reason_predictions}/{processed_count})")
    print(f"Combined Action-Reason Prediction Accuracy: {action_reason_accuracy:.2f}% ({correct_action_reason_predictions}/{processed_count})")
else:
    print("No valid image-text pairs were processed.")
print("---------------------------------------------")

# --- Function to Calculate and Print Scores ---
def print_scores_user_format(category_name, results_list, total_data_len):
    print(f"\n--- [PART 2] Ground Truth Similarity Scores for: {category_name} ---")
    valid_probabilities = [r["probability"] for r in results_list if r.get("probability") is not None]

    if valid_probabilities:
        average_probability = sum(valid_probabilities) / len(valid_probabilities)
        judgement_result_ = sum(1 for p in valid_probabilities if p > 0.5)
        print(f"Average similarity probability over {len(valid_probabilities)} successfully processed pairs: {average_probability:.4f}")
        print(f"Match percent is {judgement_result_ / total_data_len * 100:.2f} %")
    else:
        print("No valid pairs were processed for this category.")
    print("----------------------------------------------------------" + "-" * len(category_name))

# Print the similarity probability scores
print_scores_user_format("Image-Action", results_action_prob, len(data_to_process))
print_scores_user_format("Image-Reason", results_reason_prob, len(data_to_process))
print_scores_user_format("Image-Action-Reason", results_action_reason_prob, len(data_to_process))